# get checkpoint

In [1]:
import config 
from models.ssakt import SSAKT
from dataset import get_dataloaders

import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, roc_auc_score

import os
import numpy as np
import pandas as pd
from datetime import datetime
import wandb

class SAKTModel(pl.LightningModule):
  def __init__(self,model_args,model="ssakt"):
    super().__init__()
    self.model = SSAKT(**model_args)
      
  def forward(self,exercise,category,tag,chapter,test,response,etime,\
    assessmentItemID_answer_mean,user_category_acc,test_elapsed,testId_answer_mean,\
      assessmentItemID_answer_sum,assessmentItemID_test_elapsed_mean,\
        user_testId_acc,user_category_mean_telapsed,user_category_correct_answer,\
          testId_answer_sum,testId_test_elapsed_mean,user_testId_mean_telapsed,\
            user_acc,chapter_answer_mean,user_chapter_acc,user_category_cum_telapsed,\
              tag_answer_mean,user_category_total_answer,tag_answer_sum,\
                chapter_answer_sum,user_correct_answer,last_prob,\
                  user_tag_acc,num_tags_per_test,category_answer_mean):
    return self.model(exercise,category,tag,chapter,test,response,etime,\
      assessmentItemID_answer_mean,user_category_acc,test_elapsed,testId_answer_mean,\
        assessmentItemID_answer_sum,assessmentItemID_test_elapsed_mean,\
          user_testId_acc,user_category_mean_telapsed,user_category_correct_answer,\
            testId_answer_sum,testId_test_elapsed_mean,user_testId_mean_telapsed,\
              user_acc,chapter_answer_mean,user_chapter_acc,user_category_cum_telapsed,\
                tag_answer_mean,user_category_total_answer,tag_answer_sum,\
                  chapter_answer_sum,user_correct_answer,last_prob,\
                    user_tag_acc,num_tags_per_test,category_answer_mean)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters())
  
  def training_step(self,batch,batch_idx):
    inputs,target_ids,target,last_question = batch
    output = self(inputs["input_ids"],inputs["input_cat"],inputs["input_tag"],inputs["input_chap"],inputs["input_test"],target_ids,inputs["input_rtime"],
                  inputs["input_assessmentItemID_answer_mean"],inputs["input_user_category_acc"],inputs["input_test_elapsed"],
                  inputs["input_testId_answer_mean"], inputs["input_assessmentItemID_answer_sum"], inputs["input_assessmentItemID_test_elapsed_mean"],
                  inputs["input_user_testId_acc"], inputs["input_user_category_mean_telapsed"], inputs["input_user_category_correct_answer"],
                  inputs["input_testId_answer_sum"], inputs["input_testId_test_elapsed_mean"], inputs["input_user_testId_mean_telapsed"],
                  inputs["input_user_acc"], inputs['input_chapter_answer_mean'], inputs["input_user_chapter_acc"], inputs['input_user_category_cum_telapsed'],
                  inputs["input_tag_answer_mean"], inputs["input_user_category_total_answer"], inputs["input_tag_answer_sum"],
                  inputs["input_chapter_answer_sum"], inputs["input_user_correct_answer"], inputs["input_last_prob"],
                  inputs["input_user_tag_acc"], inputs["input_num_tags_per_test"], inputs["input_category_answer_mean"])
    target_mask = (target_ids != 0)
    output = torch.masked_select(output.squeeze(),target_mask)
    target = torch.masked_select(target,target_mask)
    loss = nn.BCEWithLogitsLoss()(output.float(),target.float())
    return {"loss":loss,"output":output,"target":target}

  def training_epoch_end(self, training_ouput):
      output = np.concatenate([i["output"].cpu().detach().numpy()
                              for i in training_ouput]).reshape(-1)
      target = np.concatenate([i["target"].cpu().detach().numpy()
                                  for i in training_ouput]).reshape(-1)
      print("train output length", len(output), output.shape)
      accuracy = accuracy_score(target, [1 if x > 0.5 else 0 for x in output])
      auc = roc_auc_score(target, output)
      self.print("train auc: ", auc, " acc: ", accuracy)
      self.log("train_auc", auc)
      self.log("train_acc", accuracy)
  
  def validation_step(self,batch,batch_idx):
    inputs,target_ids,target,last_question = batch
    output = self(inputs["input_ids"],inputs["input_cat"],inputs["input_tag"],inputs["input_chap"],inputs["input_test"],target_ids,inputs["input_rtime"],
                  inputs["input_assessmentItemID_answer_mean"],inputs["input_user_category_acc"],inputs["input_test_elapsed"],
                  inputs["input_testId_answer_mean"], inputs["input_assessmentItemID_answer_sum"], inputs["input_assessmentItemID_test_elapsed_mean"],
                  inputs["input_user_testId_acc"], inputs["input_user_category_mean_telapsed"], inputs["input_user_category_correct_answer"],
                  inputs["input_testId_answer_sum"], inputs["input_testId_test_elapsed_mean"], inputs["input_user_testId_mean_telapsed"],
                  inputs["input_user_acc"], inputs['input_chapter_answer_mean'], inputs["input_user_chapter_acc"], inputs['input_user_category_cum_telapsed'],
                  inputs["input_tag_answer_mean"], inputs["input_user_category_total_answer"], inputs["input_tag_answer_sum"],
                  inputs["input_chapter_answer_sum"], inputs["input_user_correct_answer"], inputs["input_last_prob"],
                  inputs["input_user_tag_acc"], inputs["input_num_tags_per_test"], inputs["input_category_answer_mean"])
    target_mask = (last_question == -1)
    output = torch.masked_select(output.squeeze(),target_mask)
    target = torch.masked_select(target,target_mask)
    loss = nn.BCEWithLogitsLoss()(output.float(),target.float())
    return {"val_loss":loss,"output":output,"target":target}

  def validation_epoch_end(self, validation_ouput):
      output = np.concatenate([i["output"].cpu().detach().numpy()
                            for i in validation_ouput]).reshape(-1)
      target = np.concatenate([i["target"].cpu().detach().numpy()
                               for i in validation_ouput]).reshape(-1)
      print("val output length", len(output), output.shape)
      accuracy = accuracy_score(target, [1 if x > 0.5 else 0 for x in output])
      auc = roc_auc_score(target, output)
      self.print("val auc: ", auc, " acc: ", accuracy)
      self.log("val_auc", auc)
      self.log("val_acc", accuracy)

  def predict_step(self,batch,batch_idx):
      inputs,target_ids,target,last_question = batch
      output = self(inputs["input_ids"],inputs["input_cat"],inputs["input_tag"],inputs["input_chap"],inputs["input_test"],target_ids,inputs["input_rtime"],
                    inputs["input_assessmentItemID_answer_mean"],inputs["input_user_category_acc"],inputs["input_test_elapsed"],
                    inputs["input_testId_answer_mean"], inputs["input_assessmentItemID_answer_sum"], inputs["input_assessmentItemID_test_elapsed_mean"],
                    inputs["input_user_testId_acc"], inputs["input_user_category_mean_telapsed"], inputs["input_user_category_correct_answer"],
                    inputs["input_testId_answer_sum"], inputs["input_testId_test_elapsed_mean"], inputs["input_user_testId_mean_telapsed"],
                    inputs["input_user_acc"], inputs['input_chapter_answer_mean'], inputs["input_user_chapter_acc"], inputs['input_user_category_cum_telapsed'],
                    inputs["input_tag_answer_mean"], inputs["input_user_category_total_answer"], inputs["input_tag_answer_sum"],
                    inputs["input_chapter_answer_sum"], inputs["input_user_correct_answer"], inputs["input_last_prob"],
                    inputs["input_user_tag_acc"], inputs["input_num_tags_per_test"], inputs["input_category_answer_mean"])
      target_mask = (last_question == -1)
      output = torch.masked_select(output.squeeze(),target_mask)
      return output

In [2]:
########### TRAINING AND SAVING MODEL #######

train_loader, val_loader, test_loader = get_dataloaders()

ARGS = {"n_dims":config.EMBED_DIMS ,
            'n_encoder':config.NUM_ENCODER,
            'n_decoder':config.NUM_ENCODER,
            'enc_heads':config.ENC_HEADS,
            'dec_heads':config.ENC_HEADS,
            'total_ex':9455,
            'total_cat':1538,
            'total_tag':913,
            'total_chap':448,
            'total_responses':9455,
            'total_test':199,
            'seq_len':config.MAX_SEQ}

model_name = "ssakt"
datenow = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")

# checkpoint path
PATH = "./checkpoint/ssakt/ckpt/2022_05_12-01_05_35_AM-epoch=13-val_auc=0.78474.ckpt" 
sakt_model = SAKTModel(model=model_name,model_args=ARGS)
trainer = pl.Trainer(gpus=-1,progress_bar_refresh_rate=21) 
sakt_model.load_state_dict(torch.load(PATH)["state_dict"])
a = trainer.predict(sakt_model, val_loader)
b = trainer.predict(sakt_model, test_loader)

# get validation result
valid_output = []
valid_result = []
for i in range(len(a)):
  valid_output.extend(a[i][0].tolist())
  valid_result.extend(a[i][1].tolist())
valid_ssakt_220512 = pd.DataFrame({"y_valid":valid_output, "ssakt_preds":valid_result})

# get test prediction result
result = []
for i in range(len(b)):
  result.extend(b[i][1].tolist())

sample_submission = pd.read_csv("/opt/ml/input/data/sample_submission.csv")
sample_submission["prediction"] = result

path = "./checkpoint/"+model_name+"/submission/"

# Check whether the specified path exists or not
isExist = os.path.exists(path)

if not isExist:
  
  # Create a new directory because it does not exist 
  os.makedirs(path)
  print("The new directory is created!")

sample_submission.to_csv(path+datenow+"_"+model_name+"_epoch=13-val_auc=0.78474.csv", index=False)
valid_ssakt_220512.to_csv(path+datenow+"_"+model_name+"_valid_ssakt_220512.csv", index=False)

loading csv.....
shape of dataframe : (2526700, 35)
no. of skills : 9454
no. of categories:  1637
no. of tags:  1012
shape after exlusion: (2526700, 35)


/opt/ml/input/Knowledge-Tracing/dataset.py:329: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["last_question"] = test_df["answered_correctly"]


test_df size (260114, 36)


/opt/ml/input/Knowledge-Tracing/dataset.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["last_question"] = [-1 if is_bool else 1 for is_bool in train_df.user_id != train_df.user_id.shift(-1)]


Grouping users...
splitting
train size:  (5358,) validation size:  (1340,) test size:  (744,)
Int64Index([2105, 2931, 551, 6632, 4559, 1163, 5377, 7343, 1211, 6144], dtype='int64', name='user_id')


...

Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 54/54 [00:06<00:00,  8.35it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 31/31 [00:03<00:00,  7.83it/s]
